In [3]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


def get_historical_data(stk, start_date, end_date):
    df = yf.download(stk, start_date, end_date, interval="1d").dropna()
    return df


def group_by_time_interval(df, number_of_weeks_in_group):
    df["start"] = df.index
    df["end"] = df.index
    df["day_of_week"] = df.index.dayofweek
    df["week_of_year"] = df.index.isocalendar().week + df.index.isocalendar().year * 100+1
    df["month_of_year"] = df.index.strftime("%m-%Y")
    df["week_group"] = (df.week_of_year / number_of_weeks_in_group).apply(np.ceil)
    df = df.groupby(["week_group"]).agg(
        {"start": "first", "end": "last", "Open": "first", "Close": "last", "Volume":"sum"}
    )
    #df['volume_rank']=df.Volume.rank(pct=True).shift(1)

    df["days"] = (df["end"] - df["start"]).dt.days + 1
    df = df.set_index("start")
    df=df[df.days>10]
    return df


def trade_parameters(df, yield_perc):
    df["strike_price"] = df.Open.round(0)
    df["straddle_yield"] = yield_perc
    df["straddle_premium"] = df.strike_price * yield_perc
    



    df['abs_change']=abs(df.Close-df.Open)
    df['abs_change_pct']=abs((df.Close-df.Open)/df.Open)
    df['abs_change_roll2']=df.abs_change.rolling(2).sum()
    df['abs_change_roll3']=df.abs_change.rolling(3).sum()
    df['abs_change_roll4']=df.abs_change.rolling(4).sum()
     
    df['abs_change_rank_shift']=df.abs_change.rank(pct=True).shift(1)
    df['abs_change_rank_r2_shift']=df.abs_change_roll2.rank(pct=True).shift(1)
    df['abs_change_rank_r3_shift']=df.abs_change_roll3.rank(pct=True).shift(1)
    df['abs_change_rank_r4_shift']=df.abs_change_roll4.rank(pct=True).shift(1)

    df["profit_loss"] = df.straddle_premium - df.abs_change
    df["log_return"] = np.log((df.strike_price + df.profit_loss) / df.strike_price)
    
    df = df.iloc[1: , :]
    
    lim=0.8
    df=df[df.abs_change_rank_shift<lim]
    df=df[df.abs_change_rank_r2_shift<lim]
    df=df[df.abs_change_rank_r3_shift<lim]
    df=df[df.abs_change_rank_r4_shift<lim]
    
    return df


atm_option_data = pd.read_csv("option_data_2022-09-02_2022-08-19.csv")


start_date = "2016-01-01"
end_date = "2022-08-13"
number_of_weeks_in_group = 2
arr = []
for i in range(0, len(atm_option_data.symbol)):
    stock = atm_option_data.symbol[i]
    yield_perc = (atm_option_data.atm_call_last[i] + atm_option_data.atm_put_last[i]) / atm_option_data.strike[i]
    df = get_historical_data(stock, start_date, end_date)
    df = group_by_time_interval(df, number_of_weeks_in_group)
    df = trade_parameters(df, yield_perc)
    df.to_csv(stock + ".csv")
    log_r = np.sum(df.log_return)
    pos_log = np.sum(df[df.log_return > 0].log_return)
    neg_log = np.sum(df[df.log_return <= 0].log_return)
    pos_periods = len(df[df.log_return > 0].log_return)
    ttl_periods = len(df.log_return)
    avg_yield = np.average(df.log_return.dropna())
    max_draw_down=np.min(df.log_return)
    std=np.std(df.abs_change_pct)
    arr.append(
        [
            stock,
            atm_option_data.last_price[i],
            atm_option_data.atm_call_last[i],
            atm_option_data.atm_put_last[i],
            yield_perc,
            log_r,
            pos_log,
            neg_log,
            pos_periods,
            ttl_periods,
            avg_yield,
            max_draw_down,
            std

        ]
    )

finaldf = pd.DataFrame(
    data=arr,
    columns=[
        "stock",
        "last_price",
        "atm_call_price",
        "atm_put_price",
        "straddle_yield",
        "log_r",
        "pos_log",
        "neg_log",
        "pos_periods",
        "ttl_periods",
        "avg_yield",
        'max_draw_down',
        'std',

    ],
)
finaldf["sharpe"] = abs(finaldf.pos_log / finaldf.neg_log)
finaldf["winr"] = finaldf.pos_periods / finaldf.ttl_periods

finaldf['recover_periods']=abs(finaldf.max_draw_down)/finaldf.avg_yield
finaldf['cover_ratio']=(finaldf.straddle_yield)/finaldf['std']
#finaldf.sort_values(by="log_r", ascending=False, inplace=True)
finaldf.to_csv("big.csv")
print(finaldf[finaldf.log_r > 0])


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

/srv/conda/envs/notebook/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/srv/conda/envs/notebook/lib/python3.7/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

ValueError: 15 columns passed, passed data had 13 columns